In [2]:
%cd ../..
%pip install .
import time
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# other1 = Car on intersection,
#             facing -1 * Range(50, 135) deg relative to ego.heading

# other2 = Car on intersection,
#             facing -1 * Range(50, 135) deg relative to ego.heading

# require abs(relative heading of other1 from other2) > 100 deg
# require (distance from ego to intersectionRegion) < 10

/home/yongming/workspace/research/apperception_new_local/apperception
Processing /home/yongming/workspace/research/apperception_new_local/apperception
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for apperception: filename=apperception-0.1.0-py3-none-any.whl size=74298 sha256=fa8aefd07134bb35a7ebbf0eff537e0e353475776c38c1e8691bf9a08adfe29e
  Stored in directory: /tmp/pip-ephem-wheel-cache-gu071rel/wheels/a4/33/17/59e20ccef806baa1a68832d19de43f0fe8d640854903fd1772
Successfully built apperception
  Attempting uninstall: apperception
    Found existing installation: apperception 0.1.0
    Uninstalling apperception-0.1.0:
      Successfully uninstalled apperception-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
"""
with (
  select *
  from item_traj
  where item_traj.object_type = 'car'
) as cars

select *
from cars as t1
join cars as t2 on t1.cameraId = t2.cameraId
join Cameras as cam on t1.cameraId = Cameras.id
where
  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) >= -15 AND
  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) <= 15 AND
  DISTANCE(cam.egoTranslation, t1.centroid, cam.timestamp) < 50 AND
  viewAngle(t1.traj, cam.egoHeading, cam.ego_Translation, cam.timestamp) < 135 AND
  DISTANCE(cam.egoTranslation, t2.centroid, cam.timestamp) < 50 AND
  viewAngle(t2.traj, cam.egoHeading, cam.ego_Translation, cam.timestamp) < 135 AND
  contained(t1.traj, road_segment_type("intersection"), cam.timestamp) and
  contained(t2.traj, road_segment_type("intersection"), cam.timestamp) and
  (facingRelative(t1.heading, cam.egoHeading, cam.timestamp) < -50 ANd
  facingRelative(t1.heading, cam.egoHeading, cam.timestamp) > -135) AND
  (facingRelative(t2.heading, cam.egoHeading, cam.timestamp) < -50 AND
  facingRelative(t2.heading, cam.egoHeading, cam.timestamp) > -135) AND
  DISTANCE(cam.egoTranslation, roadSection("intersection"), cam.timestamp) < 10 AND
  (facingRelative(t1.heading, t2.heading, cam.timestamp) > 100 OR
  facingRelative(t1.heading, t2.heading, cam.timestmap) < -100)

"""

In [3]:
### Prepare the world for queries ###
from apperception.new_world import *
# directly ingest the mini dataset and boston roadnetwork if needed
# from apperception.new_db import database
# database.reset()
# from benchmarks.ingest_scenic_data import ingest_data
# ingest_data()

name = 'ScenicWorld' # world name
world = empty_world(name=name)

### Query ###
from apperception.utils import F
world = world.filter(lambda obj: obj.object_type == 'vehicle.cars')
def pred(obj1, obj2, cam):
    return (
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) >= -15 and
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) <= 15 and
        F.distance(cam.ego, obj1, cam.timestamp) < 50 and
        F.viewAngle(obj1, cam.ego, cam.timestamp) < 135 and
        F.distance(cam.ego, obj2, cam.timestamp) < 50 and
        F.viewAngle(obj2, cam.ego, cam.timestamp) < 135 and
        F.contained(obj1.traj, F.road_segment("intersection"), cam.timestamp) and
        F.contained(obj2.traj, F.road_segment("intersection"), cam.timestamp) and
        (F.facing_relative(obj1, cam.ego, cam.timestamp) < -50 or
            F.facing_relative(obj1, cam.ego, cam.timestamp) > -135) and
        (F.facing_relative(obj2, cam.ego, cam.timestamp) < -50 or
            F.facing_relative(obj2, cam.ego, cam.timestamp) > -135) and
        F.distance(cam.ego, roadSection(obj1.traj, cam.timestamp)) < 10 and
        (F.facing_relative(obj1, obj2, cam.timestamp) > 100 or
            F.facing_relative(obj1, obj2, cam.timestamp) < -100)
    )
world = world.filter("lambda obj1, obj2, cam: F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) >= -15 and " + 
        "F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) <= 15 and " + 
        "F.distance(cam.ego, obj1, cam.timestamp) < 50 and " +
        "F.view_angle(obj1, cam.ego, cam.timestamp) < 135 and " +
        "F.distance(cam.ego, obj2, cam.timestamp) < 50 and " +
        "F.view_angle(obj2, cam.ego, cam.timestamp) < 135 and " +
        "F.contained(obj1.traj, F.road_segment('intersection'), cam.timestamp) and " +
        "F.contained(obj2.traj, F.road_segment('intersection'), cam.timestamp) and " +
        "(F.facing_relative(obj1, cam.ego, cam.timestamp) < -50 or " +
            "F.facing_relative(obj1, cam.ego, cam.timestamp) > -135) and " +
        "(F.facing_relative(obj2, cam.ego, cam.timestamp) < -50 or " +
            "F.facing_relative(obj2, cam.ego, cam.timestamp) > -135) and " +
        "F.distance(cam.ego, obj1, cam.timestamp) < 10 and " +
        "F.distance(cam.ego, obj2, cam.timestamp) < 10 and " +
        "(F.facing_relative(obj1, obj2, cam.timestamp) > 100 or " +
            "F.facing_relative(obj1, obj2, cam.timestamp) < -100)")

start = time.time()

key = world.get_id_and_time(num_joined_tables=2)

end = time.time()
print(format(end-start))

execute: filter
execute: filter
execute: get_id_and_time
get_id_and_time 
        SELECT table_0.itemId,table_1.itemId, cameras.timestamp
        FROM (
        SELECT DISTINCT *
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        
        WHERE (table_0.objectType='vehicle.cars')
        ) as table_0
        JOIN (
        SELECT DISTINCT *
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        
        WHERE (table_0.objectType='vehicle.cars')
        ) as table_1 ON table_1.cameraId = table_0.cameraId
        JOIN Cameras ON Cameras.cameraId = table_0.cameraId
        WHERE ((facingRelative(Cameras.egoHeading, roadDirection(Cameras.egoTranslation, Cameras.timestamp), Cameras.timestamp)>=(-15)) AND (facingRelative(Cameras.egoHeading, roadDirection(Cameras.egoTranslation, Cameras.timestamp), Cameras.timestamp)<=15) AND (distance(Cameras.egoTranslation, table_0.trajCentroids, Cameras.timestamp)<50) AND (viewAngle(table_0.trajCentro